In [1]:
from functions import get_data
import pandas as pd
import numpy as np


In [2]:
us_data=get_data('GOLD')

In [3]:
us_data['mtime']=pd.to_datetime(us_data['time'],unit='s')
us_data = us_data[us_data['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]
us_data.reset_index(drop=True, inplace=True)

In [4]:
numpy_data=us_data[['time','Open', 'High', 'Low', 'Close']].values

In [5]:
test_numpy = numpy_data[:10000]


In [6]:
from functions import calc_prof,evaluate

In [7]:
def x_points(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    t=val_h['mtime'].time()
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    
def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
def find_status(data,index,window,size):
    cspreads=[]
    # print(index,window,size)
    # print("fstatus here1")
    for i in range(window):
        datapoint=data.iloc[index-i]
        close=datapoint['Close']
        openv=datapoint['Open']
        cspread=abs(close-openv)
        cspreads.append(cspread)
    # print("fstatus here2")
    if max(cspreads)>size:
        return 1
    else:
        return 0
    
def daily(data,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    distcounter=0
    statuscounter=0
    distcounter2=0
    statuscounter2=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {adj_time}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            # print(ind)
            begin_max_value,begin_min_value=x_points(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                       
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        distcounter+=1
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)
                            statuscounter+=1
                        # elif find_status(data,i,window,c_size)==0:
                            
                            # continue
                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                       
                        
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        distcounter2+=1
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
                            statuscounter2+=1
                        # elif find_status(data,i,window,c_size)==0:
                            
                            # continue
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades#,distcounter,statuscounter,distcounter2,statuscounter2
                    
                           
def daily_move2(data,adjtime, lim, window, c_size, sl_adj, multiplier ):
    name=f'starttime:{adjtime,} window: {window}candlesize{c_size}sl_adj{sl_adj}multiplier{multiplier}'
    result = daily(data, adjtime, lim, window, c_size, sl_adj, multiplier)
    print('i am here')
    evaluated_info=evaluate(result,name)
    print('i am here2')
    result_dict1 = {
    'Strategy': name,
    'Take_profit_level':"level1(tp1)",
    'Max_Profit': evaluated_info[0],
    'Num_Buy_Trades': evaluated_info[1],
    'Total_Buy_Profit': evaluated_info[2],
    'Num_Sell_Trades': evaluated_info[3],
    'Total_Sell_Profit': evaluated_info[4],
    'Max_Profit_Index': evaluated_info[5],
    'Index_of_Max_Profit': evaluated_info[6],
    'Max_Duration': evaluated_info[7],
    'ID_Max_Duration': evaluated_info[8],
    'total_profitable_trades':evaluated_info[9],
    'total_unprofitable_trades':evaluated_info[10],
    'total_profitable_trades':evaluated_info[11],
    'total_loss_trades':evaluated_info[12]}

    return result_dict1
    # with open(name, 'w', newline='') as csvfile:
    #     fieldnames = result_dict1.keys()
    #     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    #     writer.writeheader()
    #     writer.writerow(result_dict1)
               
 


In [8]:
import cProfile

In [9]:
# us_data2 = us_data[us_data['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]

testing_dt=us_data[:10000]
# testing_dt.reset_index(drop=True, inplace=True)


In [10]:
def distance_checker2(price, value, lim):
    return int(abs(price - value) > lim)

In [11]:
import numpy as np

In [12]:
import datetime

In [13]:
def x_points_numpy(data,index):
    
    val_h=data[index,:]
    value=val_h[0]

    datetime_object = np.datetime64('1970-01-01T00:00:00') + np.array(value, dtype='timedelta64[s]')

    date_part = datetime_object.astype('datetime64[D]')
    unix_timestamps = (date_part.astype('datetime64[s]') - np.datetime64('1970-01-01T00:00:00')).astype(int)
    # print(str(date_part))
    # print(unix_timestamps)
    # print(f'original{value}')
    datetime1 = datetime.datetime.utcfromtimestamp(value).strftime('%Y-%m-%d %H:%M:%S UTC')
    datetime2 = datetime.datetime.utcfromtimestamp(unix_timestamps).strftime('%Y-%m-%d %H:%M:%S UTC')

# Output the results
    # print("Original timestamp :", datetime1)
    # print("Modified :", datetime2)
    # filtered_data = data[(data['mtime'].dt.date == date_part) & (data.index <= index)]
    filtered_data = data[(data[:, 0].astype('datetime64[s]').astype('datetime64[D]') >= date_part) & (data[:, 0].astype('datetime64[s]').astype('datetime64[D]') <= np.datetime64(datetime.datetime.utcfromtimestamp(unix_timestamps).strftime('%Y-%m-%d')))]
    # print(len(filtered_data))
    max_value = np.max(filtered_data[:, 4])  # Assuming the values are in the second column
    min_value = np.min(filtered_data[:, 4])

    return max_value,min_value


In [14]:
def x_points(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    # print(start_date)
    t=val_h['mtime'].time()
    # print(t)
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(len(filtered_data))
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value

In [15]:
def find_status2(data,index,window,size):
    cspreads=[]
    # print(index,window,size)
    # print("fstatus here1")
    for i in range(window):
        datapoint=data[index-i,:]
        close=datapoint[4]
        openv=datapoint[1]
        cspreads.append(abs(close-openv))
    # print("fstatus here2")
    if max(cspreads)>size:
        return 1
    else:
        return 0

In [16]:
def convert_unix_to_datetime(unix_timestamp):
    # Convert Unix timestamp to datetime object
    dt_object = datetime.datetime.utcfromtimestamp(unix_timestamp)
    
    # Format datetime object as a string
    formatted_datetime = dt_object.strftime('%Y-%m-%dT%H:%M:%S')

    return formatted_datetime


In [17]:
def daily2(data,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    distcounter=0
    statuscounter=0
    distcounter2=0
    statuscounter2=0
    
    for i in range(len(data)):
        val_h = data[i, :]
        time = val_h[0]

        datetime_object = np.datetime64('1970-01-01T00:00:00') + np.array(time, dtype='timedelta64[s]')
        date_part = datetime_object.astype('datetime64[D]')  # Use datetime64[s] for both date and time

        # Combine date and time
        combined_datetime_str = f"{date_part} {adjtime}"
        combined_datetime = np.datetime64(combined_datetime_str)

        # Convert datetime to Unix timestamp
        check_timestamp = int(combined_datetime.astype('datetime64[s]').astype(int))
        # unix_timestamps = (date_part - np.datetime64('1970-01-01T00:00:00')).astype(int)

        # adj_time = np.datetime64(adjtime).astype(int)
        
        if time <= check_timestamp:
            # print(combined_datetime)
            # ba=convert_unix_to_datetime(time)
            # ab=convert_unix_to_datetime(check_timestamp)
            # print(f'disallowed first time{ba}')
            # print(f'diaaslowedd upper time {ab}')
            continue
        else:
            # print('yyyyyyy')
            # print(i)
            maxv,minv=x_points_numpy(data,i)
            price=val_h[4]
            print(f'price {price},maxvalue{maxv}')
            # print(f'price {price},minvalue{minv}')
            if trade==0:
                if price>maxv:
                    print("i am true")
                    if distance_checker2(price,maxv,lim)==0:
                        maxv=price
                        
                        continue
                    elif distance_checker2(price,maxv,lim)==1:
                        # print('i am here 4')
                        # distcounter+=1
                        if find_status2(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            
                            trade= 1
                            openprice=price
                            opendate=val_h[0]
                            lowp=val_h[3]
                            print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # statuscounter+=1
                            # print(trade)
                        # elif find_status2(data,i,window,c_size)==0:
                           
                            # continue
                            
                elif price<minv:
                    if distance_checker2(price,minv,lim)==0:
                        minv=price
                        
                        continue
                    elif distance_checker2(price,minv,lim)==1:
                        # print('i am here3 ')
                        # distcounter2+=1
                        if find_status2(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade= -1
                            openprice=price
                            # print(trade)
                            opendate=val_h[0]
                            print('i am here2 ')
                            highp=val_h[4]
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
                            # statuscounter2+=1
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h[0] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h[0]
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h[0]
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h[0]
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades#,distcounter,statuscounter,distcounter2,statuscounter2

In [18]:
yg=daily2(numpy_data,'03:00:00',2.3,2,2.5,2.5,5)
print(len(yg))

price 1191.99,maxvalue1206.48
price 1191.24,maxvalue1206.48
price 1191.37,maxvalue1206.48
price 1190.51,maxvalue1206.48
price 1192.62,maxvalue1206.48
price 1193.42,maxvalue1206.48
price 1193.02,maxvalue1206.48
price 1193.03,maxvalue1206.48
price 1193.19,maxvalue1206.48
price 1193.27,maxvalue1206.48
price 1192.77,maxvalue1206.48
price 1192.14,maxvalue1206.48
price 1191.98,maxvalue1206.48
price 1191.44,maxvalue1206.48
price 1191.54,maxvalue1206.48
price 1191.95,maxvalue1206.48
price 1192.45,maxvalue1206.48
price 1194.45,maxvalue1206.48
price 1196.41,maxvalue1206.48
price 1195.93,maxvalue1206.48
price 1195.53,maxvalue1206.48
price 1195.46,maxvalue1206.48
price 1196.39,maxvalue1206.48
price 1196.01,maxvalue1206.48
price 1196.85,maxvalue1206.48
price 1196.52,maxvalue1206.48
price 1195.56,maxvalue1206.48
price 1195.88,maxvalue1206.48
price 1195.67,maxvalue1206.48
price 1195.39,maxvalue1206.48
price 1193.48,maxvalue1206.48
price 1193.81,maxvalue1206.48
price 1194.82,maxvalue1206.48
price 1194

price 1196.59,maxvalue1206.48
price 1195.15,maxvalue1206.48
price 1195.76,maxvalue1206.48
price 1196.31,maxvalue1206.48
price 1196.0,maxvalue1206.48
price 1197.79,maxvalue1206.48
price 1198.74,maxvalue1206.48
price 1200.74,maxvalue1206.48
price 1200.74,maxvalue1206.48
price 1203.5,maxvalue1206.48
price 1203.62,maxvalue1206.48
price 1202.47,maxvalue1206.48
price 1203.37,maxvalue1206.48
price 1203.15,maxvalue1206.48
price 1202.17,maxvalue1206.48
price 1203.92,maxvalue1206.48
price 1204.72,maxvalue1206.48
price 1205.22,maxvalue1206.48
price 1206.48,maxvalue1206.48
price 1205.67,maxvalue1206.48
price 1205.14,maxvalue1206.48
price 1206.03,maxvalue1206.48
price 1203.91,maxvalue1206.48
price 1203.75,maxvalue1206.48
price 1202.91,maxvalue1220.65
price 1203.4,maxvalue1220.65
price 1204.0,maxvalue1220.65
price 1204.23,maxvalue1220.65
price 1204.27,maxvalue1220.65
price 1204.58,maxvalue1220.65
price 1204.75,maxvalue1220.65
price 1204.98,maxvalue1220.65
price 1205.4,maxvalue1220.65
price 1205.29,m

KeyboardInterrupt: 

In [19]:
import MetaTrader5 as mt5

In [20]:
def fget_data(currency):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2023, 1, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

In [21]:
nowd=fget_data('GOLD')


In [22]:
nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
nowd = nowd[nowd['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]
nowd.reset_index(drop=True, inplace=True)
numpy_=nowd[['time','Open', 'High', 'Low', 'Close']].values


In [55]:
test1=daily2(numpy_data,'03:00:00',3.0,4,2.5,3.0,3)
test2=daily2(numpy_data,'03:00:00',4.0,4,2.5,3.0,3)
test3=daily2(numpy_data,'03:00:00',5.0,4,2.5,3.0,3)
test4=daily2(numpy_data,'03:00:00',6.0,4,2.5,3.0,3)
test5=daily2(numpy_data,'03:00:00',3.5,4,2.5,3.0,3)
test6=daily2(numpy_data,'03:00:00',7.0,4,2.5,3.0,3)


In [ ]:
vtest1=daily2(numpy_data,'03:00:00',3.0,4,2.5,5.0,4)
vtest2=daily2(numpy_data,'03:00:00',4.0,7,3.5,3.0,3)
vtest3=daily2(numpy_data,'03:00:00',5.0,3,4.5,3.0,3)
vtest4=daily2(numpy_data,'03:00:00',6.0,10,5.5,3.0,3)
vtest5=daily2(numpy_data,'03:00:00',3.5,6,2.5,3.0,3)
vtest6=daily2(numpy_data,'03:00:00',7.0,8,2.5,3.0,3)


In [60]:
vtest11=daily2(numpy_data,'03:00:00',3.0,4,5.5,4.0,5)
vtest12=daily2(numpy_data,'03:00:00',4.0,7,4.5,3.0,4)
vtest13=daily2(numpy_data,'03:00:00',5.0,3,2.5,3.0,6)
vtest14=daily2(numpy_data,'03:00:00',6.0,10,2.5,3.0,2)
vtest15=daily2(numpy_data,'03:00:00',3.5,6,2.5,3.0,2)
vtest16=daily2(numpy_data,'03:00:00',7.0,8,2.5,3.0,2)


In [47]:
h,bn,mn,vc,xz=daily2(test_numpy,'03:00:00',2.3,2,2.5,2.5,5)
h

9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309


[('Buy', 1421860500.0, 1421936100.0, -3.7899999999999636),
 ('Buy', 1425355200.0, 1425362400.0, 13.490000000000009),
 ('Buy', 1426773600.0, 1426855500.0, 12.889999999999873),
 ('Buy', 1432831500.0, 1433175300.0, 17.519999999999982)]

In [83]:
nwt

[]

In [ ]:
cProfile.run("daily(testing_dt,'03:00:00',2.3,2,2.5,2.5,5)")

In [21]:
f=daily(nowd,'03:00:00',2.3,7,2.5,2.5,5)

In [38]:
ht=daily(nowd,'03:00:00',1.5,10,1.5,1.5,6)
print('okay3')
ht1=daily(nowd,'03:00:00',1.5,10,1.5,1.5,3)
ht2=daily(nowd,'03:00:00',1.5,10,1.5,1.5,4)
print('okay2')
ht3=daily(nowd,'03:00:00',1.5,10,1.5,1.5,7)

print('okay1')
ht5=daily(nowd,'03:00:00',1.5,10,1.5,1.5,8)

okay3
okay2
okay1


In [23]:
ht21=daily(nowd,'03:00:00',1.5,10,1.5,1.5,5)
ht22=daily(nowd,'03:00:00',1.5,10,1.5,1.5,3)
ht23=daily(nowd,'03:00:00',1.5,10,1.5,1.5,2)
ht24=daily(nowd,'03:00:00',1.5,10,1.5,1.5,6)
ht25=daily(nowd,'03:00:00',1.5,10,1.5,1.5,7)
ht26=daily(nowd,'03:00:00',1.5,10,1.5,1.5,8)




In [24]:
ht2=daily(nowd,'03:00:00',1.5,10,1.5,1.5,4)


In [ ]:
kl4=daily(nowd,'03:00:00',1.5,10,1.5,1.5,2)

In [24]:
t1=daily(nowd,'03:00:00',2.0,7,3.5,3.5,3)
t2=daily(nowd,'03:00:00',3.0,7,3.5,3.5,2)
t3=daily(nowd,'03:00:00',3.0,7,2.5,2.5,2)
t4=daily(nowd,'03:00:00',4.0,3,3.0,3.5,3)
t5=daily(nowd,'03:00:00',5.0,7,5.5,4.5,3)
t6=daily(nowd,'03:00:00',2.3,7,3.5,2.5,4)
t7=daily(nowd,'03:00:00',2.3,7,5.5,3.5,3)
t8=daily(nowd,'03:00:00',2.3,7,3.0,4.5,2)
t9=daily(nowd,'03:00:00',2.3,7,2.5,3.5,4)
t10=daily(nowd,'03:00:00',2.3,7,2.5,5.0,2)

In [27]:
t11 = daily(nowd, '03:00:00', 4.12, 3, 3.45, 4.21, 3)
t21 = daily(nowd, '03:00:00', 2.98, 5, 4.12, 3.77, 2)
t31 = daily(nowd, '03:00:00', 5.55, 6, 2.33, 4.01, 2)
t41 = daily(nowd, '03:00:00', 3.74, 2, 4.01, 3.21, 3)
t51 = daily(nowd, '03:00:00', 5.03, 7, 5.12, 4.89, 3)
t61 = daily(nowd, '03:00:00', 2.33, 4, 4.12, 3.33, 4)
t71 = daily(nowd, '03:00:00', 4.55, 6, 3.21, 4.01, 3)
t81 = daily(nowd, '03:00:00', 3.01, 7, 2.98, 3.21, 2)
t91= daily(nowd, '03:00:00', 4.45, 5, 3.45, 2.77, 4)
t101 = daily(nowd, '03:00:00', 2.77, 3, 2.33, 4.55, 2)


In [26]:
tt1 = daily(nowd, '03:00:00', 4.12, 3, 3.45, 4.21, 3)
tt2 = daily(nowd, '03:00:00', 2.98, 5, 4.12, 3.77, 2)
tt3 = daily(nowd, '03:00:00', 5.55, 6, 2.33, 4.01, 2)
tt4 = daily(nowd, '03:00:00', 3.74, 2, 4.01, 3.21, 3)
tt5 = daily(nowd, '03:00:00', 5.03, 7, 5.12, 4.89, 3)
tt6 = daily(nowd, '03:00:00', 2.33, 4, 4.12, 3.33, 4)
tt7 = daily(nowd, '03:00:00', 4.55, 6, 3.21, 4.01, 3)
tt8 = daily(nowd, '03:00:00', 3.01, 7, 2.98, 3.21, 2)
tt9 = daily(nowd, '03:00:00', 4.45, 5, 3.45, 2.77, 4)
tt10 = daily(nowd, '03:00:00', 2.77, 3, 2.33, 4.55, 2)


In [25]:
print(len(t1))

1260


In [34]:
import csv

csv_file = 'Dailystrat.csv'

# Write the data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['TradeType', 'OpenDate', 'CloseDate', 'Profit'])
    writer.writerows(ht21)

print(f"CSV file '{csv_file}' has been created.")


CSV file 'Dailystrat.csv' has been created.


In [33]:
gg=evaluate(ht21,'')

The total profit for   was :255.6800000000087
the trade that took most time took 3 days 03:45:00 and made -3.9200000000000728 type:Buy
Buy :1621 profit:239.00999999999772
Sell:2863 profit:16.670000000010987
the max duration was 3 days 03:45:00
The trade with the highest profit made 17.470000000000027 pips:1747.0000000000027 


In [32]:
def create_dict_and_df(evaluated_info, name="YourStrategyName"):
    result_dict1 = {
        'Strategy': name,
        'Take_profit_level': "level1(tp1)",
        'Max_Profit': evaluated_info[0],
        'Num_Buy_Trades': evaluated_info[1],
        'Total_Buy_Profit': evaluated_info[2],
        'Num_Sell_Trades': evaluated_info[3],
        'Total_Sell_Profit': evaluated_info[4],
        'Max_Profit_Index': evaluated_info[5],
        'Index_of_Max_Profit': evaluated_info[6],
        'Max_Duration': evaluated_info[7],
        'ID_Max_Duration': evaluated_info[8],
        'total_profitable_trades': evaluated_info[9],
        'total_unprofitable_trades': evaluated_info[10],
        'total_profitable_trades': evaluated_info[11],
        'total_loss_trades': evaluated_info[12]
    }

    # Create a DataFrame from the dictionary
    result_df = pd.DataFrame(result_dict1, index=[0])

    return result_dict1, result_df


In [88]:
ko,po=create_dict_and_df(gg)
po

,Strategy,Take_profit_level,Max_Profit,Num_Buy_Trades,Total_Buy_Profit,Num_Sell_Trades,Total_Sell_Profit,Max_Profit_Index,Index_of_Max_Profit,Max_Duration,ID_Max_Duration,total_profitable_trades,total_unprofitable_trades,total_loss_trades
0,YourStrategyName,level1(tp1),204.74,752,236.0,1280,-31.26,18.96,948,3 days 15:00:00,974,2378.9,1088,-2174.16


In [31]:
print(len(f),len(h))

267 4
